In [31]:
%pip install --upgrade tensorflow


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Вадим\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import tensorflow as tf
from transformers import TFBertModel
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model

In [33]:
df = pd.read_csv('banks.csv', delimiter='\t',  skipinitialspace=True, engine="python")
X = df['Text']
y = df['Score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [34]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

X_train_tokens = tokenizer(X_train.to_list(), padding=True, truncation=True, return_tensors="tf")
X_test_tokens = tokenizer(X_test.to_list(), padding=True, truncation=True, return_tensors="tf")


In [35]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [36]:
input_ids = Input(shape=(X_train_tokens['input_ids'].shape[1],), dtype=tf.int32, name='input_ids')
input_masks = Input(shape=(X_train_tokens['attention_mask'].shape[1],), dtype=tf.int32, name='attention_mask')

bert_output = bert_model([input_ids, input_masks])[0]
x = GlobalAveragePooling1D()(bert_output)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[input_ids, input_masks], outputs=output)

In [37]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


In [47]:
history = model.fit(x={'input_ids': X_train_tokens['input_ids'], 'attention_mask': X_train_tokens['attention_mask']}, y=y_train, batch_size=32, epochs=5, validation_split=0.1)


Epoch 1/5


UnimplementedError: Graph execution error:

Detected at node binary_crossentropy/Cast defined at (most recent call last):
  File "c:\Users\Вадим\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main

  File "c:\Users\Вадим\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code

  File "C:\Users\Вадим\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\Вадим\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1053, in launch_instance

  File "C:\Users\Вадим\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\Вадим\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Users\Вадим\AppData\Local\Programs\Python\Python310\lib\asyncio\base_events.py", line 595, in run_forever

  File "c:\Users\Вадим\AppData\Local\Programs\Python\Python31

In [43]:
input_ids = Input(shape=(X_train_tokens['input_ids'].shape[1],), dtype=tf.int32, name='input_ids')
input_masks = Input(shape=(X_train_tokens['attention_mask'].shape[1],), dtype=tf.int32, name='attention_mask')

bert_output = bert_model([input_ids, input_masks])[0]  #получение выхода первого слоя BERT
x = GlobalAveragePooling1D()(bert_output)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=[input_ids, input_masks], outputs=output)

In [44]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


In [45]:
history = model.fit(x={'input_ids': X_train_tokens['input_ids'], 'attention_mask': X_train_tokens['attention_mask']}, y=y_train, batch_size=32, epochs=5, validation_split=0.1)


Epoch 1/5


UnimplementedError: Graph execution error:

Detected at node binary_crossentropy/Cast defined at (most recent call last):
  File "c:\Users\Вадим\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main

  File "c:\Users\Вадим\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code

  File "C:\Users\Вадим\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\Вадим\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1053, in launch_instance

  File "C:\Users\Вадим\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\Вадим\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Users\Вадим\AppData\Local\Programs\Python\Python310\lib\asyncio\base_events.py", line 595, in run_forever

  File "c:\Users\Вадим\AppData\Local\Programs\Python\Python31

In [46]:
loss, accuracy = model.evaluate(x={'input_ids': X_test_tokens['input_ids'], 'attention_mask': X_test_tokens['attention_mask']}, y=y_test)
print(f'точность: {accuracy:.2f}')

UnimplementedError: Graph execution error:

Detected at node binary_crossentropy/Cast defined at (most recent call last):
  File "c:\Users\Вадим\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main

  File "c:\Users\Вадим\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code

  File "C:\Users\Вадим\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\Вадим\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1053, in launch_instance

  File "C:\Users\Вадим\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\Вадим\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Users\Вадим\AppData\Local\Programs\Python\Python310\lib\asyncio\base_events.py", line 595, in run_forever

  File "c:\Users\Вадим\AppData\Local\Programs\Python\Python31